In [1]:
import torch
import os
import albumentations as A

from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from dataset import PlatesDataset, collate_fn
from fasterrcnn_trainer import ModelTrainer

/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
## Опеределяем конфигурацию обучения

Config = {
    # detection train dataset params
    'root_path': '../vkcv2022-contest-02-carplates/data/',
    'json_path': '../vkcv2022-contest-02-carplates/data/train.json',
    'val_size': 0.2,
    
    # detection training params
    'model_name': 'faster_rcnn_v6',
    'device': 'cuda:2',
    'num_epochs': 20,
    'lr': 3e-4,
    'batch_size': 16,
    'num_workers': 8
}

In [6]:
## Добавляем аугментацию для расширения датасета

train_transform = A.Compose([A.HorizontalFlip(p=0.25),
                            A.Rotate(limit=(-20, 20), p=0.25),
                            A.ColorJitter(brightness=0.3, contrast=0.2, saturation=0.2, hue=0.05, p=0.25)],
                            bbox_params={
                                        'format': 'pascal_voc',
                                        'label_fields': ['labels']
                                        })

trainset = PlatesDataset(root_path=Config['root_path'], json_path=Config['json_path'],
                         sample_type='train', val_size=Config['val_size'], transform=train_transform)

valset = PlatesDataset(root_path=Config['root_path'], json_path=Config['json_path'],
                       sample_type='val', val_size=Config['val_size'])

print('Train size:', len(trainset))
print('Val size:', len(valset))

Train size: 20506
Val size: 5127


In [7]:
## Запускаем обучение модели

faster_rcnn_model = fasterrcnn_resnet50_fpn_v2(pretrained=True)
in_features = faster_rcnn_model.roi_heads.box_predictor.cls_score.in_features
faster_rcnn_model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)

    
for param in faster_rcnn_model.backbone.body.parameters():
    param.requires_grad = False
    
for param in faster_rcnn_model.backbone.body.layer4.parameters():
    param.requires_grad = True
    
for name, param in faster_rcnn_model.named_parameters():
    print(name, param.requires_grad)
    

faster_rcnn_model.to(Config['device'])
optimizer = torch.optim.AdamW(faster_rcnn_model.parameters(), lr=Config['lr'])


train_loader = DataLoader(trainset, batch_size=Config['batch_size'], shuffle=True, 
                          collate_fn=collate_fn, num_workers=Config['num_workers'])
val_loader = DataLoader(valset, batch_size=Config['batch_size'], shuffle=False, 
                        collate_fn=collate_fn, num_workers=Config['num_workers'])


scheduller = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, max_lr=Config['lr'], 
                                                 steps_per_epoch=len(train_loader), 
                                                epochs=Config['num_epochs'], pct_start=0.05, anneal_strategy='cos')


model_trainer = ModelTrainer(model=faster_rcnn_model, model_name=Config['model_name'], 
                             train_loader=train_loader, val_loader=val_loader,
                            optimizer=optimizer, scheduller=scheduller, device=Config['device'], val_classes=[1])

model_trainer.train_net(num_epochs=Config['num_epochs'])

backbone.body.conv1.weight False
backbone.body.bn1.weight False
backbone.body.bn1.bias False
backbone.body.layer1.0.conv1.weight False
backbone.body.layer1.0.bn1.weight False
backbone.body.layer1.0.bn1.bias False
backbone.body.layer1.0.conv2.weight False
backbone.body.layer1.0.bn2.weight False
backbone.body.layer1.0.bn2.bias False
backbone.body.layer1.0.conv3.weight False
backbone.body.layer1.0.bn3.weight False
backbone.body.layer1.0.bn3.bias False
backbone.body.layer1.0.downsample.0.weight False
backbone.body.layer1.0.downsample.1.weight False
backbone.body.layer1.0.downsample.1.bias False
backbone.body.layer1.1.conv1.weight False
backbone.body.layer1.1.bn1.weight False
backbone.body.layer1.1.bn1.bias False
backbone.body.layer1.1.conv2.weight False
backbone.body.layer1.1.bn2.weight False
backbone.body.layer1.1.bn2.bias False
backbone.body.layer1.1.conv3.weight False
backbone.body.layer1.1.bn3.weight False
backbone.body.layer1.1.bn3.bias False
backbone.body.layer1.2.conv1.weight False


1it [00:02,  2.99s/it]

Global Loss: 1.4458268880844116
Object Loss: 0.693958044052124
Reg Loss: 0.01906971074640751


21it [00:38,  1.86s/it]

Global Loss: 0.8208143711090088
Object Loss: 0.6799728331111726
Reg Loss: 0.013976888731122017


...

Global Loss: 0.06575417561866974
Object Loss: 0.00884102504696476
Reg Loss: 0.006217855235790613


1184it [34:48,  1.85s/it]

Cant open image


Process Process-30:
Process Process-29:
Process Process-28:
Process Process-31:
Process Process-32:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/multiprocessing/util.py", line 360, in _exit_function
    _run_finalizers()
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3.9/multiprocessing/process.py", line 318, in _bootstrap
    util._exit_function()
  File "/home/ndubrovnyi/.conda/envs/venv/lib/python3

RuntimeError: DataLoader worker (pid 2575712) exited unexpectedly with exit code 1. Details are lost due to multiprocessing. Rerunning with num_workers=0 may give better error trace.

Обучили модель деткции Faster_rcnn на 20 эпох:

Лучшая метрика MAP50_95 = 77.9%     

Логи обучения: 'Logs/faster_rcnn_v5.txt'